In [4]:
from rdflib import URIRef, Literal, Graph, Namespace
from rdflib.namespace import FOAF, RDF, RDFS, XSD, DC
import urllib
from datetime import datetime
from SPARQLWrapper import SPARQLWrapper, JSON, N3

In [18]:
print(XSD)

http://www.w3.org/2001/XMLSchema#


In [15]:
# Define SPARQL query

sparql_query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dc:  <http://purl.org/dc/elements/1.1/>

    Select distinct ?scientist ?name ?birthdate ?description ?thumbnail WHERE {
        ?scientist rdf:type dbo:Scientist ;           
            rdfs:label ?name ;
            dbo:birthDate ?birthdate; 
            dct:description ?description .
     FILTER ((lang(?name)="en") && (lang(?description)="en") 
                 && (SUBSTR(STR(?birthdate),6)=SUBSTR(STR(bif:curdate('')),6)) 
                 && (STRLEN(STR(?birthdate))>6)) .
     OPTIONAL {?scientist dbo:thumbnail ?thumbnail }.
    }
"""

In [16]:
sparql = SPARQLWrapper("https://dbpedia.org/sparql")
sparql.setQuery(sparql_query)

sparql.setReturnFormat(JSON)
results = sparql.query().convert()

In [17]:
results

{'head': {'link': [],
  'vars': ['scientist', 'name', 'birthdate', 'description', 'thumbnail']},
 'results': {'distinct': False,
  'ordered': True,
  'bindings': [{'scientist': {'type': 'uri',
     'value': 'http://dbpedia.org/resource/Frank_Drake'},
    'name': {'type': 'literal', 'xml:lang': 'en', 'value': 'Frank Drake'},
    'birthdate': {'type': 'typed-literal',
     'datatype': 'http://www.w3.org/2001/XMLSchema#date',
     'value': '1930-05-28'},
    'description': {'type': 'literal',
     'xml:lang': 'en',
     'value': 'American astronomer and astrophysicist'},
    'thumbnail': {'type': 'uri',
     'value': 'http://commons.wikimedia.org/wiki/Special:FilePath/Frank_Drake_-_edit.jpg?width=300'}},
   {'scientist': {'type': 'uri',
     'value': 'http://dbpedia.org/resource/Varadaraja_V._Raman'},
    'name': {'type': 'literal',
     'xml:lang': 'en',
     'value': 'Varadaraja V. Raman'},
    'birthdate': {'type': 'typed-literal',
     'datatype': 'http://www.w3.org/2001/XMLSchema#dat

In [7]:
# Define further Namespaces
EX = Namespace("https://ex1.org/")
DBO = Namespace("http://dbpedia.org/ontology/")

# create a graph
g = Graph()

# bind vocabularies to PREFIXES
g.bind('foaf', FOAF)
g.bind('dbo', DBO)
g.bind('dc', DC)

In [14]:
# Explore JSON structure to retrieve results first!

for result in results["results"]["bindings"]:
        
    scientist_uri = result["scientist"]["value"]
    scientist_name = result['name']['value']
    scientist_birthdate = result['birthdate']['value']
    scientist_description = result['description']['value']
    
    print(URIRef(scientist_uri))
    
    # add triples to graph g
    g.add((URIRef(scientist_uri), RDF.type, DBO.Scientist))
    g.add((URIRef(scientist_uri), FOAF.name, Literal(scientist_name,lang='en')))
    g.add((URIRef(scientist_uri), DBO.birthdate, Literal(scientist_birthdate,datatype=XSD.date)))
    g.add((URIRef(scientist_uri), DC.description, Literal(scientist_description,lang='en')))
    
    try: #because OPTIONAL in SPARQL query
        scientist_thumbnail = result['thumbnail']['value']
        g.add((URIRef(scientist_uri), DBO.thumbnail, URIRef(scientist_thumbnail)))
    except:
        print('No thumbnail for:', scientist_name)
        

# print(g.serialize(format="n3").decode("utf-8"))

# store graph g in an N3 serialized file
g.serialize(destination='../output_data/example_outpudata.n3', format='n3')

http://dbpedia.org/resource/Robert_Stirling_Newall
http://dbpedia.org/resource/John_Coleman_Moore
No thumbnail for: John Coleman Moore
http://dbpedia.org/resource/K._Sridhar
No thumbnail for: K. Sridhar
http://dbpedia.org/resource/Edward_A._Irving
No thumbnail for: Edward A. Irving
http://dbpedia.org/resource/Derek_Denton
No thumbnail for: Derek Denton
http://dbpedia.org/resource/Anthony_A._Hyman
http://dbpedia.org/resource/Franciscus_Donders
http://dbpedia.org/resource/John_Wishart_(surgeon)
http://dbpedia.org/resource/André_Lussier
http://dbpedia.org/resource/Bernard_Dwork
No thumbnail for: Bernard Dwork
http://dbpedia.org/resource/Herbert_Gursky
No thumbnail for: Herbert Gursky
http://dbpedia.org/resource/Bernard_M._Oliver
No thumbnail for: Bernard M. Oliver
http://dbpedia.org/resource/Wolfgang_Ostwald
No thumbnail for: Wolfgang Ostwald
http://dbpedia.org/resource/Alan_W._Bishop
No thumbnail for: Alan W. Bishop
http://dbpedia.org/resource/Alexander_G._Petrov
http://dbpedia.org/resou

<br>

## Create HTML output

In [10]:
# Create HTML output

# s = ""
# for i in range(n):
#     s+=str(i)

html_web_page = ""

print('<html><head><title>Scientific Birthdays of Today</title></head>')
html_web_page+=('<html><head><title>Scientific Birthdays of Today</title></head>')

#extract Weekday %A / Month %B / Day of the Month %d by formatting today's date accordingly
datum = datetime.today().strftime("%A  %B %d")
print('<body><h1>Scientific Birthdays of {}</h1>'.format(datum))
html_web_page+=('<body><h1>Scientific Birthdays of {}</h1>'.format(datum))

print('<ul>')
html_web_page+=('<ul>')


#result in results["results"]["bindings"]:
        

for result in results["results"]["bindings"]:
    
    if ("scientist" in result):
        #Create Wikipedia Link
        url = ("http://en.wikipedia.org/wiki/" + result["scientist"]["value"].split('/')[-1]).encode('ascii', 'ignore')
    else:
        url = 'NONE'  
        
    if ("name" in result):
        name = result['name']['value'].encode('ascii', 'ignore')
    else:
        name = 'NONE'
    
    if ("birthdate" in result):
        date = result["birthdate"]["value"].encode('ascii', 'ignore')
    else:
        date = 'NONE'        
        
    if ("description" in result):
        description = result["description"]["value"].encode('ascii', 'ignore')
    else:
        description = ' '  
    
    if ("thumbnail" in result):
        pic = result["thumbnail"]["value"]#.encode('ascii', 'ignore')
    else:
        pic = 'http://upload.wikimedia.org/wikipedia/commons/7/7a/Question_Mark.gif'


    #parse date as datetime
    dt = datetime.strptime(date.decode('ascii'), '%Y-%m-%d')
    
    # print the results in list
    print('<li><b>{}</b> -- <img src="{}" height="60px"> <a href="{}">{}</a>, {} </li>'.format(dt.year, 
                                                                                               pic.replace("300px", "60px"), 
                                                                                               url.decode('ascii'), 
                                                                                               name.decode('ascii'), 
                                                                                               description.decode('ascii')))
    html_web_page+=('<li><b>{}</b> -- <img src="{}" height="60px"> <a href="{}">{}</a>, {} </li>'.format(dt.year, 
                                                                                                         pic.replace("300px", "60px"), 
                                                                                                         url.decode('ascii'), 
                                                                                                         name.decode('ascii'), 
                                                                                                         description.decode('ascii')))

print('</ul>')
html_web_page+=('</ul>')
print('</body></html>')
html_web_page+=('</body></html>')

# save to html file
text_file = open("../output_data/example_output_application.html", "w")
text_file.write(html_web_page)
text_file.close()

<html><head><title>Scientific Birthdays of Today</title></head>
<body><h1>Scientific Birthdays of Wednesday  May 27</h1>
<ul>
<li><b>1812</b> -- <img src="http://en.wikipedia.org/wiki/Special:FilePath/Robert_Stirling_Newall.jpg?width=300" height="60px"> <a href="http://en.wikipedia.org/wiki/Robert_Stirling_Newall">Robert Stirling Newall</a>, Scottish engineer and astronomer </li>
<li><b>1923</b> -- <img src="http://upload.wikimedia.org/wikipedia/commons/7/7a/Question_Mark.gif" height="60px"> <a href="http://en.wikipedia.org/wiki/John_Coleman_Moore">John Coleman Moore</a>, American mathematician </li>
<li><b>1961</b> -- <img src="http://upload.wikimedia.org/wikipedia/commons/7/7a/Question_Mark.gif" height="60px"> <a href="http://en.wikipedia.org/wiki/K._Sridhar">K. Sridhar</a>, Indian scientist </li>
<li><b>1927</b> -- <img src="http://upload.wikimedia.org/wikipedia/commons/7/7a/Question_Mark.gif" height="60px"> <a href="http://en.wikipedia.org/wiki/Edward_A._Irving">Edward A. Irving</a

In [34]:
import numpy as np
mat = np.zeros((10, 10))
np.cos(mat)

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])